<h1>СБОРНЫЙ ПРОЕКТ - 2. Интернет-магазин продуктов питания.<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Описание-проекта" data-toc-modified-id="Описание-проекта-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Описание проекта</a></span></li><li><span><a href="#Описание-данных" data-toc-modified-id="Описание-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Описание данных</a></span></li><li><span><a href="#Ход-работы" data-toc-modified-id="Ход-работы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Ход работы</a></span><ul class="toc-item"><li><span><a href="#Загрузка-данных-и-изучение-информации" data-toc-modified-id="Загрузка-данных-и-изучение-информации-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Загрузка данных и изучение информации</a></span></li><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Изучение-и-проверка-данных" data-toc-modified-id="Изучение-и-проверка-данных-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Изучение и проверка данных</a></span></li><li><span><a href="#Изучение-воронки-событий" data-toc-modified-id="Изучение-воронки-событий-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Изучение воронки событий</a></span></li><li><span><a href="#Изучение-результатов-эксперимента" data-toc-modified-id="Изучение-результатов-эксперимента-3.5"><span class="toc-item-num">3.5&nbsp;&nbsp;</span>Изучение результатов эксперимента</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# Описание проекта

Интернет-магазин продуктов питания хочет разобраться как ведут себя пользователи мобильного приложения.

Предлагается:

- изучить воронку продаж

- узнать как пользователи доходят до покупки

- узнать сколько пользователей доходят до покупки

- выяснить на каких этапах теряются пользователи

- исследовать результаты А/А/В-эксперимента: дизайнеры предложили поменять шрифт

- выяснить, какой шрифт лучше


# Описание данных

Логи пользователей мобильного приложения в файле logs_exp.csv

EventName — название события;

DeviceIDHash — уникальный идентификатор пользователя;

EventTimestamp — время события;

ExpId — номер эксперимента: 246 и 247 — контрольные группы, а 248 — экспериментальная.



# Ход работы

## Загрузка данных и изучение информации

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from plotly import graph_objects as go
import plotly.express as px
import datetime as dt
import numpy as np
from math import floor
import math as mth
from scipy import stats as st
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', -1)

In [2]:
try:
    
    df = pd.read_csv('logs_exp.csv', delimiter = "\t") 
except:
    
    df = pd.read_csv('/datasets/logs_exp.csv', delimiter = "\t")
    

FileNotFoundError: [Errno 2] No such file or directory: '/datasets/logs_exp.csv'

In [ ]:
df.head()

In [ ]:
df.info()

## Подготовка данных

In [ ]:
df = df.rename(columns={'EventName':'event_name', 'DeviceIDHash':'device_id',
                        'EventTimestamp':'event_timestamp', 'ExpId':'exp_id'
                       })
print(df.columns)

Переименовали столбцы и привели их названия к стандартам

In [ ]:
df['event_time'] = pd.to_datetime(df['event_timestamp'] , unit='s')
df['event_time'].head()

Перевели тип данных  времени в нужный формат и поместили эти данные в новый столбец

In [ ]:
df['date'] = pd.to_datetime(df['event_time'].dt.date)
df['date'].head()

Добавили отдельный столбец с датами

## Изучение и проверка данных

In [ ]:
display(df.duplicated().sum())
round(df.duplicated().sum() / len(df)*100, 2)

Посчитали количество явных дубликатов. Во всем файле получилось 417 полных дубликатов, така как это всего 0.17% - можем их удалить:

In [ ]:
df = df.drop_duplicates().reset_index(drop=True)

In [ ]:
df.head()

In [ ]:
df.info()

Получили таблицу на 6 солонок и 243713 строк очищенных от дубликатов. Пропуски отсутствуют.

In [ ]:
df['event_name'].unique()

Выяснили, что в таблице содержится всего 5 уникальных событий, которые совершают пользователи.

In [ ]:
df['device_id'].mean()

В логе всего 243713 событий

In [ ]:
df['device_id'].nunique()

И 7551 уникальный пользователь

In [ ]:
plt.figure(figsize=(8,5))
sns.histplot(data=df.groupby('device_id')[['event_name']].count(),x='event_name')
plt.title('Количество событий на пользователя')
plt.xlabel('Количество событий')
plt.ylabel('Количество пользователей')
plt.xlim(0,250);

In [ ]:
df.groupby('device_id')['event_name'].agg('count').describe().round(1)

В среднем на одного пользователя приходится 32 события

In [ ]:
df['event_time'].describe()

Проверим, данными за какой период мы обладаем. Нам предоставлены данные с 25 июля 2019 года по 7 августа 2019 года.

In [ ]:
sns.set_style("whitegrid")
plt.hist(data=df, x="event_time")
plt.xticks(rotation=30)
plt.xlabel('Дата')
plt.ylabel('Количество')
plt.title('Количество логов по дням');

По гистограмме видно, что не во все даты мы имеем полные данные, визуально можно определить, что наиболее полные данные появляются только с 1 августа 2019 года. Для дальнейшей работы можно отбросить более ранний период, не обладающий полной информацией,однако, сначала проверим какое количество данных мы потеряем. Безболезненно мы можем отбросить 1% данных, для этого посчитаем какая дата будет являться первым перцентилем:

In [ ]:
date_full=np.percentile(df['date'], 1)

In [ ]:
new_df=df.query('date>@date_full')

In [ ]:
new_df['date'].describe()

Действительно, получилось, что во все дни до 1 августа 2019 года было получено всего 1% данных. Дальше будем изучать только оствашийся период: с 1 августа 2019 года по 7 августа 2019 года.

In [ ]:
print('Процент потеряных событий:', round((1-new_df['event_name'].count()/df['event_name'].count())*100,2),'%')


In [ ]:
print('Процент потеряных пользователей:', round((1-new_df['device_id'].nunique()/df['device_id'].nunique())*100,2),'%')


При актуализации периода для изучения данных мы потеряли 1.16% событий и 0.23% пользователей, что является не существенным.

In [ ]:
new_df.groupby('device_id')['exp_id'].nunique().reset_index().query('exp_id > 1')

Проверили, есть ли пользователи, которые попали в более чем одну группу тестирования. Таких пользователей не обнаружено.

In [ ]:
new_df['exp_id'].unique()

В таблице после обработки и актуализации периода остались пользователи всех трех групп. 

## Изучение воронки событий

In [ ]:
new_df.groupby('event_name')['device_id'].agg({'count', 'nunique'}).sort_values(by='count',ascending=False).reset_index()


Посчитали, сколько раз встречалось каждое событие. Упорядочили по убыванию количества повторений. Самое частое событие: MainScreenAppear - 117328 повторений и 7419  уникальных пользователей приходится на это событие. OffersScreenAppear	- имеет 46333 повторений на	4593 уникальных пользователя, CartScreenAppear повторяется	42303 раз и встречается у	3734 пользователей, PaymentScreenSuccessful	повторяется 33918 раз, количество уникальных пользователей - 3539, самое редкое событие: Tutorial(1005 раз и 840 пользователей), скорее всего оно не является этапом который участвует в процессе покупки товара, скорее всего это инструкция по работе с мобильным приложением, подавляющее большенство пользователей в нем не были заинтересованы.	

In [ ]:

funnel = (new_df.
          groupby('event_name').
          agg({'device_id': 'nunique'}).
          reset_index().
          rename(columns={'device_id' : 'total_users'}).
          sort_values(by='total_users', ascending=False))
funnel['percent'] = (funnel['total_users'] / new_df['device_id'].nunique() * 100).round(0)
funnel['prev_step'] = funnel['total_users'].shift()
funnel['step_conv'] = (funnel['total_users'] / funnel['prev_step'] * 100).round(0)
funnel

Удалив событие, не имеющее прямого отношения к покупке товара, можем предположить в какой последовательности происходили события: 

1. MainScreenAppear - появление главного экрана

2. OffersScreenAppear - демонстрация товара (экран с товаром)

3. CartScreenAppear - переход в корзину

4. PaymentScreenSuccessful - экран успешной оплаты заказа.

По логике, событие Tutorial должно было идти сразу за появлением главного экрана


In [ ]:
funnel_visual=new_df.query('event_name != "Tutorial"').groupby('event_name')['device_id'].nunique().sort_values(ascending=False).reset_index()

fig=go.Figure(go.Funnel( y=funnel_visual['event_name'], x=funnel_visual['device_id'], 
                        textposition='inside', textinfo='value+percent previous+percent initial',opacity=0.85))
fig.update_layout(title_text='Воронка событий')
fig.show()

Построив воронку событий мы получили следующие результаты:
    
    - Первое событие(открытие главного экрана приложения) совершили 7419 пользователей (все уникальные пользователи из рассматриваемых данных).
    
    - Второе событие(демонстрация товара) совершили уже 4593 пользователя, что составляет 62% от тех, которые совершили первый шаг.
    
    - Третье событие(переход в корзину) совершили еще меньше пользователей: 3734, что составляет 50% от всех пользователей, открывших главный экран приложения и 81% от просмотревших товар.
    
    - Последнее событие(экран успешной оплаты) был у 3539 пользователей, это 48% от совершивших первый шаг и 95% от перешедших в корзину.

По полученным результатам можно сделать вывод, что больше всего пользователей теряется на этапе перехода к демонстрации товара, возможно необходимо размещать мотивирующие к переходу для просмотра товара позиции, акционные предложения и прочее. 38% пользователей не совершают этот переход. 

Самые маленькие потери пользователей на событии Экран успешной оплаты, т.е. из всех, перешедших в корзину, только 5% не совершают оплату. 

48% пользователей доходят от первого события до оплаты. Не плохая конверсия, но есть зона для роста.
    

## Изучение результатов эксперимента

In [ ]:

new_df = new_df.query('event_name != "Tutorial"')
users_group = new_df.groupby('exp_id')['device_id'].nunique()
users_group['246+247'] = users_group[246] + users_group[247]
users_group

Посчитали количество пользователей в каждой контрольной группе: Группа 246 - 2483 пользователя, Группа 247 - 2512 пользователей, Группа 248 - 2535 пользователей. Создали  еще одну контрольную группу 246+247, объединив группы 246 и 247. В ней получилось 4995 пользователей.

In [ ]:
groups=new_df.pivot_table(index='event_name', columns='exp_id', values='device_id',
                          aggfunc='nunique').sort_values(by=246,ascending=False)
groups['246+247'] = groups[246] + groups[247]
groups['miss_part246'] = ((1 - groups[246] / users_group[246]) * 100).round(1)
groups['miss_part247'] = ((1 - groups[247] / users_group[247]) * 100).round(1)
groups['miss_part248'] = ((1 - groups[248] / users_group[248]) * 100).round(1)
groups["miss_part246+247"] = ((1 - groups['246+247'] / users_group['246+247'])  * 100).round(1)

groups.reset_index(inplace=True)
groups

В каждой группе есть не большой процент(1.3-1.7%) поьзователей, не совершивших первое событие, скорее всего это часть пользователей, оставшихся после корректировки периода. 

Создадим воронку событий по каждой контрольной группе:

In [ ]:
fig_groups = go.Figure()
fig_groups.add_trace(go.Funnel(name='246', y=groups['event_name'], x=groups[246], 
                        textposition='inside', textinfo='value+percent previous+percent initial'))
fig_groups.add_trace(go.Funnel(name='247', y=groups['event_name'], x=groups[247], 
                        textposition='inside', textinfo='value+percent previous+percent initial'))
fig_groups.add_trace(go.Funnel(name='248', y=groups['event_name'], x=groups[248], 
                        textposition='inside', textinfo='value+percent previous+percent initial'))
fig_groups.add_trace(go.Funnel(name='246+247', y=groups['event_name'], x=groups['246+247'], 
                        textposition='inside', textinfo='value+percent previous+percent initial'))
fig_groups.update_layout(title_text='Воронка событий в разрезе тестовых групп')

 Самое популярное событие - MainScreenAppear, в гуппе 246 это событие совершили 2450 пользователей, в группе 247 - 2476 пользователей, в группе 248 -  2493 пользователя, в объединенной группе 246+247 - 4926 пользователей. 

Судя по воронке продаж, данные всех  групп очень близки по значениям.

В А/В-тестировании проверяем гипотезу о равенстве выборок, но сначала проверим находят ли статистические критерии разницу между выборками 246 и 247 (А/А-тест).Используем Z-критерий (статистический тест, позволяющий определить, различаются ли два средних значения генеральной совокупности, когда дисперсии известны и размер выборки велик). Для удобства, напишем функцию.

In [ ]:
def z_test(group1, group2, alpha): 
    p1_event = groups.loc[event_name, group1]
    p2_event = groups.loc[event_name, group2] 
    p1_users = users_group.loc[group1] 
    p2_users = users_group.loc[group2] 
    
    
    p1 = p1_event / p1_users 
    p2 = p2_event / p2_users 
    
    p_combined = (p1_event + p2_event) / (p1_users + p2_users) 
    difference = p1 - p2
    
    z_value = difference / mth.sqrt(p_combined * (1 - p_combined) * (1 / p1_users + 1 / p2_users))

    distr = st.norm(0, 1)
    p_value = (1 - distr.cdf(abs(z_value))) * 2
    
    bonferroni_alpha = alpha / 16
    
    print('Тестирование для групп {} и {}, событие: {}, p-значение: {p_value:.2f}'.format(group1, group2, event_name, p_value=p_value))
    if (p_value < bonferroni_alpha):
        print("Отвергаем нулевую гипотезу о равенстве данных в группах")
    else:
        print("Не получилось отвергнуть нулевую гипотезу о равенстве данных в группах")

Зададим гипотезы для наших тестов:

H0: Статистически значимой разницы между группами нет

H1: Статистически значимая разница между группами есть

In [ ]:
for event_name in groups.index:
    z_test(246, 247, 0.01)
    print()


Статистически значимой разницы между контрольными  группами 246 и 247 не выявлено.

In [ ]:
for event_name in groups.index:
    z_test(246, 248, 0.01)
    print()

Для первой контрольной группы и группы с измененным шрифтом статистически значимой разницы не обнаружено.

In [ ]:
for event_name in groups.index:
    z_test(247, 248, 0.01)
    print()

Для второй контрольной группы и группы с измененным шрифтом статистически значимой разницы не найдено.

In [ ]:
for event_name in groups.index:
    z_test('246+247', 248, 0.01)
    print()

Для объединенной контрольной группы и группы с измененным шрифтом статистически значимой разницы также не обнаружено. 

# Выводы

Для интернет-магазина продуктов было проведено исследование. Целью исследования было изучить воронку продаж, узнать как пользователи доходят до покупки, узнать сколько пользователей доходят до покупки, выяснить на каких этапах теряются пользователи, исследовать результаты А/А/В-эксперимента: дизайнеры предложили поменять шрифт
выяснить, какой шрифт лучше. 

В ходе работы  стало видно, что не во все даты имеются полные данные, получилось определить, что наиболее полные данные появляются только с 1 августа 2019 года. Были отброшены данные за более ранний период, не обладающий полной информацией, количество  потеряенных данных составило 1%, в их числе 1.16% событий и 0.23% пользователей. Актуальный период: с 1 августа 2019 года по 7 августа 2019 года.

Выяснилось, что самое частое событие: MainScreenAppear - 117328 повторений и 7419 уникальных пользователей приходится на это событие. OffersScreenAppear - имеет 46333 повторений на 4593 уникальных пользователя, CartScreenAppear повторяется 42303 раз и встречается у 3734 пользователей, PaymentScreenSuccessful повторяется 33918 раз, количество уникальных пользователей - 3539, самое редкое событие: Tutorial(1005 раз и 840 пользователей), скорее всего оно не является этапом который участвует в процессе покупки товара, возможно, это инструкция по работе с мобильным приложением, подавляющее большенство пользователей в нем не были заинтересованы.

Последовательность событий, скорее всего, следующая:

1. MainScreenAppear - появление главного экрана

2. Tutorial - инструкция 

3. OffersScreenAppear - демонстрация товара (экран с товаром)

4. CartScreenAppear - переход в корзину

5. PaymentScreenSuccessful - экран успешной оплаты заказа.

Для построения воронки событий мы удалили событие Tutorial, так как оно не имеет отношения к процессу покупки и получили  следующие результаты:

- Первое событие(открытие главного экрана приложения) совершили 7419 пользователей (все уникальные пользователи из рассматриваемых данных).

- Второе событие(демонстрация товара) совершили уже 4593 пользователя, что составляет 62% от тех, которые совершили первый шаг.

- Третье событие(переход в корзину) совершили еще меньше пользователей: 3734, что составляет 50% от всех пользователей, открывших главный экран приложения и 81% от просмотревших товар.

- Последнее событие(экран успешной оплаты) был у 3539 пользователей, это 48% от совершивших первый шаг и 95% от перешедших в корзину.

По полученным результатам можно сделать вывод, что больше всего пользователей теряется на этапе перехода к демонстрации товара, возможно необходимо размещать мотивирующие к переходу для просмотра товара позиции, акционные предложения и прочее. 38% пользователей не совершают этот переход.
Самые маленькие потери пользователей на событии "Экран успешной оплаты", т.е. из всех, перешедших в корзину, только 5% не совершают оплату.
48% пользователей доходят от первого события до оплаты. Не плохая конверсия, но есть зона для роста.

Для изучения результатов А/А/В теста посчитали количество пользователей в каждой контрольной группе: Группа 246 - 2483 пользователя, Группа 247 - 2512 пользователей, Группа 248 - 2535 пользователей. Создали еще одну контрольную группу 246+247, объединив группы 246 и 247. В ней получилось 4995 пользователей.

В каждой группе есть не большой процент(1.3-1.7%) поьзователей, не совершивших первое событие, скорее всего это часть пользователей, оставшихся после корректировки периода.

Самое популярное событие - MainScreenAppear, в гуппе 246 это событие совершили 2450 пользователей, в группе 247 - 2476 пользователей, в группе 248 -  2493 пользователя, в объединенной группе 246+247 - 4926 пользователей. 

Судя по воронке событий, данные всех  групп очень близки по значениям. И можно сказать, что разбиение на группы работает корректно.

В А/В-тестировании проверли гипотезу о равенстве выборок, но сначала проверили, находят ли статистические критерии разницу между выборками 246 и 247 (А/А-тест).Использовали Z-критерий (статистический тест, позволяющий определить, различаются ли два средних значения генеральной совокупности, когда дисперсии известны и размер выборки велик).

Гипотезы для проверки: 

H0: Статистически значимой разницы между группами нет


H1: Статистически значимая разница между группами есть

При проверке статистических гипотез  выбрали уровень значимости равный 1%. Мы сделали 16 проверок статистических гипотез, при таком количестве возникает вероятность ошибки ложноположительного результата. Для того, чтобы групповая вероятность ошибки в данном случае не превышала определенный уровень значимости α (в нашем случае - 0.01), была применена поправка Бонферрони к коэффициенту статистической значимости.

Статистически значимой разницы между группами во всех тестах выявлено не было.
Значит, изменение шрифта не повлияло ни на один этап воронки продаж. Опасения менеджеров, что пользователям будет непривычно не оправдались. Изменения шрифтов для всех пользователей можно применять либо оставить все как есть и искать иные гипотезы для улучшения конверсии и повышения бизнес-показателей.